<a href="https://colab.research.google.com/github/hyeon9698/2019_undergraduate_student_autonomous_car_sensor/blob/master/week_11/%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_%EC%BA%90%EA%B8%80_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.6MB 8.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Using cached kaggle-1.5.6.tar.gz (58 kB)
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=5924dac8f041a6b9ea0b32d45b5549ba75365f78555d5bd85679be514afa612e
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [3]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle -v

Kaggle API 1.5.6


In [5]:
!kaggle competitions download -c regression-cabbage-price

  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 25.3MB/s]


In [6]:
!unzip regression-cabbage-price.zip

Archive:  regression-cabbage-price.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [11]:
import numpy as np
import torch
xy_train = np.loadtxt('train_cabbage_price.csv', delimiter=',', dtype=np.float32, skiprows=1, usecols=range(1,6))
x_data = torch.from_numpy(xy_train[:,0:-1])
y_data = torch.from_numpy(xy_train[:,[-1]])
xy_test = np.loadtxt('test_cabbage_price.csv', delimiter=',', dtype=np.float32, skiprows=1, usecols=range(1,5))
test_x_data = torch.from_numpy(xy_test)

In [12]:
print(x_data)
print(y_data)
print(test_x_data)

tensor([[ -4.9000, -11.0000,   0.9000,   0.0000],
        [ -3.1000,  -5.5000,   5.5000,   0.8000],
        [ -2.9000,  -6.9000,   1.4000,   0.0000],
        ...,
        [  2.9000,  -2.1000,   8.0000,   0.0000],
        [  2.9000,  -1.6000,   7.1000,   0.6000],
        [  2.1000,  -2.0000,   5.8000,   0.4000]])
tensor([[2123.],
        [2123.],
        [2123.],
        ...,
        [2901.],
        [2901.],
        [2901.]])
tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])


In [27]:
epochs = 10001
lr = 0.0005
w = torch.randn(1,4, requires_grad=True)
b = torch.randn(1, requires_grad=True)

def model (x):
    return x @ w.t() + b

def cost_fn(prediction, true):
    diff = prediction - true
    return torch.sum(diff * diff)/diff.numel()

for epoch in range(epochs):
    pred = model(x_data)
    cost = cost_fn(pred, y_data)
    cost.backward()
    if epoch%100 == 0:
        print("Epoch: {}, Cost: {}".format(epoch, cost.item()))
    with torch.no_grad():
        w -= w.grad*lr
        b -= b.grad*lr
        w.grad.zero_()
        b.grad.zero_()
x_test = torch.tensor([-.7,-6.6,2.,0.1])
y_test = model(x_test)
print("Prediction : {}".format(y_test.item()))

Epoch: 0, Cost: 12635817.0
Epoch: 100, Cost: 2298501.75
Epoch: 200, Cost: 2271830.25
Epoch: 300, Cost: 2266016.0
Epoch: 400, Cost: 2260465.75
Epoch: 500, Cost: 2254997.0
Epoch: 600, Cost: 2249607.0
Epoch: 700, Cost: 2244294.75
Epoch: 800, Cost: 2239058.25
Epoch: 900, Cost: 2233896.75
Epoch: 1000, Cost: 2228809.0
Epoch: 1100, Cost: 2223793.25
Epoch: 1200, Cost: 2218849.0
Epoch: 1300, Cost: 2213975.0
Epoch: 1400, Cost: 2209169.5
Epoch: 1500, Cost: 2204432.0
Epoch: 1600, Cost: 2199761.0
Epoch: 1700, Cost: 2195155.75
Epoch: 1800, Cost: 2190615.25
Epoch: 1900, Cost: 2186138.0
Epoch: 2000, Cost: 2181723.25
Epoch: 2100, Cost: 2177370.0
Epoch: 2200, Cost: 2173077.0
Epoch: 2300, Cost: 2168843.5
Epoch: 2400, Cost: 2164668.5
Epoch: 2500, Cost: 2160551.5
Epoch: 2600, Cost: 2156491.0
Epoch: 2700, Cost: 2152486.25
Epoch: 2800, Cost: 2148536.75
Epoch: 2900, Cost: 2144641.0
Epoch: 3000, Cost: 2140798.25
Epoch: 3100, Cost: 2137008.0
Epoch: 3200, Cost: 2133269.5
Epoch: 3300, Cost: 2129581.25
Epoch: 3400

In [16]:
prediction = model(test_x_data)

In [17]:
print(test_x_data)
print(prediction)

tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])
tensor([[3340.5764],
        [3182.0393],
        [3918.9480],
        [3282.1575],
        [3295.8901],
        [3331.3252]], grad_fn=<AddBackward0>)


In [18]:
import pandas as pd
submit = pd.read_csv('submit_sample.csv')
submit

Id  Expected
0   0      1000
1   1      1000
2   2      1000
3   3      1000
4   4      1000
5   5      1000

In [19]:
submit['Expected'][0]

1000

In [20]:
for i in range(len(prediction)):
    submit['Expected'][i] = prediction[i].item()

In [21]:
submit

Id  Expected
0   0      3340
1   1      3182
2   2      3918
3   3      3282
4   4      3295
5   5      3331

In [22]:
submit.to_csv('submit.csv',mode='w', index=False)

In [23]:
! kaggle competitions submit -c regression-cabbage-price -f submit.csv -m "first_submit"

100% 54.0/54.0 [00:02<00:00, 24.3B/s]
Successfully submitted to Regression : Cabbage price